In [ ]:
if 'google.colab' in str(get_ipython()):
  !pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
import fullcontrol as fc
from google.colab import files
!pip install trimesh

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 kB 20.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import trimesh
from math import pi, cos, sin, tan, sqrt, radians, degrees, ceil
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# prompt: get user to upload a file then automatically identify the file name assuming only one file uploaded
def upload_file_and_get_name():
    uploaded = files.upload()
    return list(uploaded.keys())[0]

# uploaded = files.upload()
# file_name = list(uploaded.keys())[0]
file_name = upload_file_and_get_name()

Saving curvey jug.STL to curvey jug.STL


In [ ]:
points_per_layer = 91 # keep this odd if you want to use zigzags that flip direction each layer to get ripple texture
num_layers = 60 #do some maths for total stl height / layer height to get this manually (will automate in future)
zigzag_value = 2 # set zigzag 'depth' (how far they stick out radially)

In [ ]:
import numpy as np, trimesh, plotly.express as px, plotly.graph_objects as go
from math import sin, cos, tan, radians, sqrt, pi

# --- Profile Extraction Functions (local (r,z)) ---

def center_mesh(mesh):
    t = -mesh.centroid; t[2] = 0; mesh.apply_translation(t); return mesh

def rotate_to_bottom(profile):
    if not profile:
        return profile
    arr = np.array(profile)
    min_index = int(np.argmin(arr[:,1]))
    return profile[min_index:] + profile[:min_index]

def filter_top_bottom(profile, tol_ratio=0.01):
    if not profile:
        return profile
    arr = np.array(profile)
    z_vals = arr[:,1]
    z_min, z_max = np.min(z_vals), np.max(z_vals)
    tol = tol_ratio * (z_max - z_min)
    bottom_idxs = [i for i, z in enumerate(z_vals) if abs(z - z_min) < tol]
    top_idxs = [i for i, z in enumerate(z_vals) if abs(z - z_max) < tol]
    def collapse_cluster(idxs):
        if not idxs:
            return None
        sub = arr[idxs]
        return idxs[int(np.argmax(sub[:,0]))]
    bottom_keep = collapse_cluster(bottom_idxs)
    top_keep = collapse_cluster(top_idxs)
    new_profile = []
    for i, pt in enumerate(profile):
        if abs(pt[1]-z_min) < tol:
            if i == bottom_keep:
                new_profile.append(pt)
        elif abs(pt[1]-z_max) < tol:
            if i == top_keep:
                new_profile.append(pt)
        else:
            new_profile.append(pt)
    return new_profile

def ensure_anticlockwise(profile):
    """
    Ensure the profile (list of (r, z) points) is in anticlockwise order.
    Uses the shoelace formula to compute the signed area.
    If area is negative, the profile is clockwise and gets reversed.
    Note: This assumes the profile is nearly closed (or sufficiently long).
    """
    if len(profile) < 3:
        return profile  # Not enough points to determine orientation.
    area = 0
    for i in range(len(profile)):
        x1, y1 = profile[i]
        x2, y2 = profile[(i+1) % len(profile)]
        area += (x1 * y2 - x2 * y1)
    if area < 0:  # Clockwise, so reverse it.
        return profile[::-1]
    return profile

def get_ordered_profile(sec, a, tol_ratio=0.01):
    if sec is None:
        return []
    if sec.entities is not None and len(sec.entities) > 0:
        try:
            inds = sec.entities[0].points
            pts = sec.vertices[inds]
        except Exception:
            pts = sec.vertices
    else:
        pts = sec.vertices
    U = np.array([cos(a), sin(a), 0])
    local_r = pts.dot(U)
    local_z = pts[:,2]
    mask = local_r > 0  # keep right-hand side only
    pts_local = np.column_stack((local_r[mask], local_z[mask]))
    profile = list(map(tuple, pts_local))

    # Enforce anticlockwise ordering immediately.
    profile = ensure_anticlockwise(profile)

    profile = rotate_to_bottom(profile)
    profile = filter_top_bottom(profile, tol_ratio)

    return profile

def process_mesh(file_path, n_slices=360, tol_ratio=0.01):
    mesh = trimesh.load_mesh(file_path)
    center_mesh(mesh)
    angles = np.linspace(0, 2*pi, n_slices, endpoint=False)
    profiles = []
    for a in angles:
        sec = mesh.section(plane_origin=mesh.centroid, plane_normal=[-sin(a), cos(a), 0])
        prof = get_ordered_profile(sec, a, tol_ratio)
        if prof:
            profiles.append((a, prof))
    return sorted(profiles, key=lambda tup: tup[0])

def local_to_global(profile, a):
    return [(r*cos(a), r*sin(a), z) for r, z in profile]

def plot_profiles(global_profiles):
    fig = go.Figure()
    for theta, prof in global_profiles:
        if len(prof) < 2:
            continue
        X, Y, Z = zip(*local_to_global(prof, theta))
        fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='lines', name=f"{theta:.2f}"))
    fig.update_layout(title="Extracted Wedge Profiles", scene=dict(aspectmode="data"))
    fig.show()

# --- Execution ---

file_path = file_name
profiles = process_mesh(file_path, n_slices=points_per_layer, tol_ratio=0.01)
# Keep only profiles with more than one point.
global_profiles = [(theta, prof) for theta, prof in profiles if len(prof) > 1]

# Plot the extracted wedge profiles.
# plot_profiles(global_profiles)

In [ ]:
# Example spiral toolpath generation from the stl profile data
# The stl profiles could be used to generate lots of different toolpath strategies like ripple texture, etc.

# --- Continuous Spiral Toolpath Generation ---

def create_continuous_spiral_toolpath(global_profiles, n_layers=10, zigzag_value=0, zigzag_percent=0):
    """
    Generates a continuous spiral toolpath.
    - Total points = n_points = n_slices * n_layers, where n_slices = len(global_profiles).
    - For each point i, fraction = i/(n_points-1) gives:
        effective_angle = fraction * (2π*n_layers)   (continuous increase)
        z = z_min + fraction(z_max - z_min)
    - The wedge profile used for interpolation is selected by computing:
          index = round((effective_angle mod 2π)/(2π) * n_slices)
    - Then, the local radius is interpolated from that wedge's profile (assumed sorted by z),
      and global coordinates are computed using the full effective_angle.
    """
    n_slices = len(global_profiles)
    if n_slices == 0:
        return []
    n_points = n_slices * n_layers + 1
    # Determine global z range from all profiles.
    z_all = [z for theta, prof in global_profiles for (r,z) in prof]
    z_min, z_max = min(z_all), max(z_all)

    spiral_path = []
    slice_now=0
    for i in range(n_points):
        fraction = i / (n_points - 1)
        eff_angle = fraction * (2*pi*n_layers)  # continuously increasing effective angle
        z_val = z_min + fraction * (z_max - z_min)
        # Compute wedge index based on effective_angle mod 2π.
        theta_wedge, prof = global_profiles[slice_now]
        prof_arr = np.array(prof)  # columns: r, z
        # Interpolate radius from the wedge's profile.
        r_val = np.interp(z_val, prof_arr[:,1], prof_arr[:,0])
        # offset every other point outwards if zagzag is required
        if i % 2 == 0:
          if zigzag_value != 0:
            r_val += zigzag_value
          elif zigzag_percent != 0:
            r_val += r_val * zigzag_percent/100
        # Use full effective_angle for global coordinates.
        x = r_val * cos(eff_angle)
        y = r_val * sin(eff_angle)
        spiral_path.append((x, y, z_val))
        slice_now = (slice_now + 1) % n_slices
    return spiral_path

# Generate a continuous spiral toolpath.
spiral_path = create_continuous_spiral_toolpath(global_profiles, n_layers=num_layers, zigzag_value=zigzag_value)
# x_sp, y_sp, z_sp = zip(*spiral_path)
steps = [fc.Point(x=point[0], y=point[1], z=point[2]) for point in spiral_path]

In [ ]:
fc.transform(steps, 'plot', fc.PlotControls(style='line'))

In [ ]:
gcode = fc.transform(steps, 'gcode')
open(f'{file_name}.gcode', 'w').write(gcode)
files.download(f'{file_name}.gcode')

   - use fc.transform(..., controls=fc.GcodeControls(printer_name='generic') to disable this message or set it to a real printer name

fc.transform guidance tips are being written to screen if any potential issues are found - hide tips with fc.transform(..., show_tips=False)
tip: set initial `extrusion_width` and `extrusion_height` in the initialization_data to ensure the correct amount of material is extruded:
   - `fc.transform(..., controls=fc.GcodeControls(initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))`



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>